In [1]:
import plotly 
plotly.tools.set_credentials_file(username='Vashanin', api_key='nd6ocZEWvaUUOLmFUEnj')

In [18]:
# https://plot.ly/~Vashanin/7/genres-and-amount-of-songs/
# https://plot.ly/~Vashanin/9/
# https://plot.ly/~Vashanin/11/
# Dashboard https://plot.ly/dashboard/Vashanin:13/view

In [16]:
import cx_Oracle
import plotly.plotly as py
import plotly.graph_objs as go
import re
import plotly.dashboard_objs as dashboard

def fileId_from_url(url):
    """Return fileId from a url."""
    raw_fileId = re.findall("~[A-z]+/[0-9]+", url)[0][1:]
    return raw_fileId.replace('/', ':')

bd_connection = cx_Oracle.connect("SYSTEM", "1111", "xe")
 
cursor = bd_connection.cursor()
 
# create plot 1   Вивести кількість пісень кожного жанру.
cursor.execute("""
SELECT 
    GENRE.GENRE_ID, 
    GENRE.GENRE_NAME, 
    COUNT(DISTINCT SONG.SONG_ID) 
FROM 
    SONG JOIN GENRE ON SONG.GENRE_ID = GENRE.GENRE_ID 
GROUP BY 
    GENRE.GENRE_ID, 
    GENRE.GENRE_NAME
""")
 
genre_names = []
amount_of_songs = []

for row in cursor:
    print("Genre: " + str(row[1]) + " and his amount of songs: " + str(row[2]))
    genre_names.append(row[1])
    amount_of_songs.append(row[2])

data = [go.Bar(
            x=genre_names,
            y=amount_of_songs
    )]
 
layout = go.Layout(
    title='Genres and amount of songs',
    xaxis=dict(
        title='Genres',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Amount of songs',
        rangemode='nonnegative',
        autorange=True,
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

fig = go.Figure(data=data, layout=layout) 
songs_genres = py.plot(fig, filename='songs-genres')
 
# create plot 2  Вивести відношення пісень виконавців.

cursor.execute("""
SELECT 
    PERFORMER.PERFORMER_ID, 
    PERFORMER.PERFORMER_NAME, 
    COUNT(DISTINCT SONG.SONG_ID) 
FROM 
    SONG JOIN PERFORMER ON SONG.PERFORMER_ID = PERFORMER.PERFORMER_ID 
GROUP BY 
    PERFORMER.PERFORMER_ID, 
    PERFORMER.PERFORMER_NAME
""")
 
performer = []
songs = []

for row in cursor:
    print("Performes " + str(row[1]) + ", songs amount: " + str(row[2]))
    performer.append(row[1])
    songs.append(row[2])

pie = go.Pie(labels=performer, values=songs)
performer_songs = py.plot([pie], filename='performer_songs')
 
# create plot 3   Вивести динаміку танців по датах
 
cursor.execute("""
SELECT
    DANCE_DATE,
    COUNT(PERSON_ID)
FROM 
    DANCE
GROUP BY 
    DANCE_DATE
""")

dance_date = []
amount_of_persons = []
 
for row in cursor:
    print("Date " + str(row[0]) + " Dancers: " + str(row[1]))
    dance_date.append(row[0])
    amount_of_persons.append(row[1])
 
 
name_memory = go.Scatter(
    x=dance_date,
    y=amount_of_persons,
    mode='lines+markers'
)
data = [name_memory]
dance_persons = py.plot(data, filename='dance_persons')
 
# --------CREATE DASHBOARD------------------ 
my_dboard = dashboard.Dashboard()
 
songs_genres_id = fileId_from_url(songs_genres)
performer_songs_id = fileId_from_url(performer_songs)
dance_persons_id = fileId_from_url(dance_persons)
 
box_1 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': songs_genres_id,
    'title': 'Genres and amount of songs.'
}
 
box_2 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': performer_songs_id,
    'title': 'Peformer song majority'
}
 
box_3 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': dance_persons_id,
    'title': 'Dance'
}
 
my_dboard.insert(box_1)
my_dboard.insert(box_2, 'below', 1)
my_dboard.insert(box_3, 'left', 2)

py.dashboard_ops.upload(my_dboard, 'My First Dashboard with Python') 

Genre: ROCK and his amount of songs: 4
Genre: HIP-HOP and his amount of songs: 2
Genre: POP and his amount of songs: 4
Performes Stone Sour, songs amount: 2
Performes Eminem, songs amount: 2
Performes Linkin Park, songs amount: 4
Performes Slipknot, songs amount: 2
Date 2018-04-27 18:33:04 Dancers: 1
Date 2018-04-27 18:04:24 Dancers: 1
Date 2018-04-14 18:03:51 Dancers: 1
Date 2018-04-09 18:04:36 Dancers: 1
Date 2018-04-02 18:04:00 Dancers: 1
Date 2018-04-02 18:32:50 Dancers: 1
Date 2018-04-01 18:04:09 Dancers: 1


'https://plot.ly/~Vashanin/13/untitled-dashboard/'